# Basic BoW model 

In [1]:
from collections import defaultdict
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from data import get_data
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def cleanup(x):
    # removes punctuation
    x = re.sub(r'[.,"!#/]+', ' ', x, flags=re.MULTILINE)
    # removs \\n
    x = re.sub(r'\\n', ' ', x, flags=re.MULTILINE)
    # removes retweets (e.g. @FER)
    x = re.sub(r'@\w+', '', x, flags=re.MULTILINE)

    return x

In [3]:
def vectorize_X(tweet, vocab):
    # obsolete 
    # tweet = re.sub(r'[.,"!#]+', ' ', tweet, flags=re.MULTILINE)
    #tweet = re.sub(r'@\w+', ' ', tweet, flags=re.MULTILINE)

    # splits tags into 2, e.g. #happy -> #, happy
    # TODO: tags could be useful for classification
    tweet_toks = word_tokenize(tweet)
    X_ = []
    for w in vocab:
        X_.append(1 if w in tweet_toks else 0)

    return X_

In [4]:
def create_vocab(tweets):
    vocab = set()
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    #stemmer = SnowballStemmer()
    
    clean_tweets = [cleanup(t) for t in tweets]
    
    for t in clean_tweets:
        for w in word_tokenize(t):
            vocab.add(w.strip())
    
    vocab -= set(stop_words)
        
    return sorted(vocab)

In [5]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [6]:
X_train, y_train = get_data('data/2018-E-c-En-train.txt')
X_test, y_test = get_data('data/2018-E-c-En-dev.txt')

X_train_clean = [cleanup(x) for x in X_train]
X_test_clean = [cleanup(x) for x in X_test]

#vocab = create_vocab(X)

vectorizer = CountVectorizer(ngram_range=(1, 4), stop_words='english')#, vocabulary=vocab)
clf = OneVsRestClassifier(LinearSVC(random_state=42))

estimators = [('vectorize', vectorizer), ('clf', clf)]

In [7]:
pipe = Pipeline(memory=None, steps=estimators).fit(X_train_clean, y_train)

print(accuracy_score(y_test, pipe.predict(X_test_clean)))

0.17042889390519186
